# SQL Server 2019 Data Virtualization - Using Polybase to query Oracle
This notebook contains an example of how to use external tables to query data in Oracle without moving data. You may need to change identity, secret, connection, database, schema, and remote table names to work with your Oracle Database.

This notebook also assumes you are using SQL Server 2019 Release Candidate or later and that the Polybase feature has been installed and enabled.

This notebook uses the sample WideWorldImporters sample database but can be used with any user database.

## Step 0: Create a database in Oracle, table, and add data

This example uses an Oracle Express Instance which by default is called XE. Run run any of these SQL statements below as a script you can use this syntax with sqlplus64

`sqlplus64 gl/glpwd@localhost:<port>/<instance> @<scriptname>.sql`

Create a user called gl with the following SQL statement

```sql
CREATE USER gl IDENTIFIED BY <password> DEFAULT TABLESPACE users TEMPORARY TABLESPACE temp QUOTA UNLIMITED ON users;
GRANT CREATE SESSION TO gl;
GRANT CREATE TABLE TO gl;
GRANT CREATE VIEW TO gl;
GRANT CREATE ANY TRIGGER TO gl;
GRANT CREATE ANY PROCEDURE TO gl;
GRANT CREATE SEQUENCE TO gl;
GRANT CREATE SYNONYM TO gl;
```

Create a table called accountsreceivable using the following SQL statement

```sql
CREATE TABLE gl.accountsreceivable (
arid int primary key,
ardate timestamp,
ardesc varchar2(100),
arref int,
aramt number(10,2)
);
```

Insert data using the following SQL statement

```sql
INSERT INTO gl.accountsreceivable VALUES (1, TO_DATE('05/31/2016', 'MM/DD/YYYY'), 'Invoice 70510', 336252, 5549.90);
```

## Step 1: Create a master key
Create a master key to encrypt the database credential

In [23]:
USE [WideWorldImporters]
GO
CREATE MASTER KEY ENCRYPTION BY PASSWORD = '<password>'
GO

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.014

## Step 2: Create a database credential.
The database credential contains the IDENTITY (login) and SECRET (password) of Oracle instance. Change this to the login and password created in Step 0.

In [24]:
CREATE DATABASE SCOPED CREDENTIAL OracleCredentials   
WITH IDENTITY = 'gl', SECRET = '<password>'
GO

Commands completed successfully.

Total execution time: 00:00:00.004

## Step 3: Create an EXTERNAL DATA SOURCE
The EXTERNAL DATA SOURCE indicates what type of data source, the connection "string", where PUSHDOWN predicates should be used (if possible), and the name of the database credential.

The LOCATION syntax is <datasourcetype>:<connection string>.

datasourcetype can be sqlserver, oracle, teradata, mongodb, or odbc (Windows only)
The connection string depends on the datasourcetype

For this example, put IP address or hostname of the Oracle instance and port number

In [25]:
CREATE EXTERNAL DATA SOURCE OracleServer
WITH ( 
LOCATION = 'oracle://<oracle host>:<port>',
PUSHDOWN = ON,
CREDENTIAL = OracleCredentials
)
GO

Commands completed successfully.

Total execution time: 00:00:00.094

## Step 4: Create a schema for the EXTERNAL TABLE
Schemas provide convenient methods to secure and organize objects

In [26]:
CREATE SCHEMA oracle
GO

Commands completed successfully.

Total execution time: 00:00:00.004

## Step 5: Create an EXTERNAL TABLE
An external table provides metadata so SQL Server knows how to map columns to the remote table. The name of the tables for the external table can be your choice. But the columns must be specified in the same order with the same name as they are defined in the remote table. Furthermore, local data types must be compatible with the remote table.

The WITH clause specifies a LOCATION. This LOCATION is different than the EXTERNAL DATA SOURCE. For Oracle, this LOCATION indicates the [instance].[schema].[table] of the Oracle table. The DATA_SOURCE clauses is the name of the EXTERNAL DATA SOURCE you created earlier.

For Oracle, the LOCATION needs to be UPPERCASE. The column names must match the target data source. The column names must be UPPERCASE for Oracle in the table definition but not when you reference them.

In [27]:
CREATE EXTERNAL TABLE oracle.accountsreceivable
(
ARID int,
ARDATE Date,
ARDESC nvarchar(100) COLLATE Latin1_General_100_BIN2_UTF8,
ARREF int,
ARAMT decimal(10,2)
)
 WITH (
 LOCATION='[XE].[GL].[ACCOUNTSRECEIVABLE]',
 DATA_SOURCE=OracleServer
)
GO

Commands completed successfully.

Total execution time: 00:00:00.275

## Step 6: Create statistics
SQL Server allows you to store local statistics about specific columns from the remote table. This can help the query processing to make more efficient plan decisions.

In [28]:
CREATE STATISTICS arrefstats ON oracle.accountsreceivable ([arref]) WITH FULLSCAN
GO

Commands completed successfully.

Total execution time: 00:00:00.227

## Step 7: Try to scan the remote table
Run a simple query on the EXTERNAL TABLE to scan all rows.

In [29]:
SELECT * FROM oracle.accountsreceivable
GO

(1 row affected)

Total execution time: 00:00:00.210

ARID,ARDATE,ARDESC,ARREF,ARAMT
1,2016-05-31,Invoice 70510,336252,5549.90


## Step 8: Query the remote table with a WHERE clause
Even though the table may be small SQL Server will "push" the WHERE clause filter to the remote table

In [30]:
SELECT * FROM oracle.accountsreceivable
WHERE arref = 336252
GO

(1 row affected)

Total execution time: 00:00:00.208

ARID,ARDATE,ARDESC,ARREF,ARAMT
1,2016-05-31,Invoice 70510,336252,5549.90


## Step 9: Join with local SQL Server tables
Find accts receivable data based on CustomerTransactionID (which matches arref in the AR tables in Oracle)

In [31]:
SELECT ct.*, oa.arid, oa.ardesc
FROM oracle.accountsreceivable oa
JOIN [Sales].[CustomerTransactions] ct
ON oa.arref = ct.CustomerTransactionID
GO

(1 row affected)

Total execution time: 00:00:00.256

CustomerTransactionID,CustomerID,TransactionTypeID,InvoiceID,PaymentMethodID,TransactionDate,AmountExcludingTax,TaxAmount,TransactionAmount,OutstandingBalance,FinalizationDate,IsFinalized,LastEditedBy,LastEditedWhen,arid,ardesc
336252,401,1,70510,NULL,2016-05-31,4826.00,723.90,5549.90,5549.90,NULL,0,3,2016-05-31 12:00:00.0000000,1,Invoice 70510
